# Project Goals:  
Create a database to store tables for the International Hurricane Watchgroup (IHW) which meets the following requirements:  
1. A table in the database should contain all the fields detailed in the csv file available here:  
https://dq-content.s3.amazonaws.com/251/storm_data.csv  
2. There should be a user created that can update, read, and insert into a table in the database.  
3. The data in the csv file mentioned above should be loaded into the table mentioned in 1.

## Read the csv file

In [8]:
import pandas as pd

df = pd.read_csv('https://dq-content.s3.amazonaws.com/251/storm_data.csv')

In [10]:
df.head(3)

,FID,YEAR,MONTH,DAY,AD_TIME,BTID,NAME,LAT,LONG,WIND_KTS,PRESSURE,CAT,BASIN,Shape_Leng
0,2001,1957,8,8,1800Z,63,NOTNAMED,22.5,-140.0,50,0,TS,Eastern Pacific,1.140175
1,2002,1961,10,3,1200Z,116,PAULINE,22.1,-140.2,45,0,TS,Eastern Pacific,1.166190
2,2003,1962,8,29,0600Z,124,C,18.0,-140.0,45,0,TS,Eastern Pacific,2.102380


## Create the sql table for the csv file  
Notes:  
* The AD_TIME column is a record of the time in Coordinated Universal Time (https://en.wikipedia.org/wiki/Coordinated_Universal_Time)  

In [184]:
import psycopg2
conn = psycopg2.connect(dbname='postgres', user='postgres')
cur = conn.cursor()

##### I'll want a way to map postgresql type codes to their descriptions to help review data types.

In [48]:
# Create a mapping dictionary:
cur.execute("SELECT oid, typname FROM pg_catalog.pg_type")
type_mappings = {}
for oid, typename in cur.fetchall():
    type_mappings[int(oid)] = typename

##### Create the table and review data types

In [59]:
# Check largest and smallest integers in the BTID columnto see if SMALLINT is an appropriate data type
print('Minimum:', df['BTID'].min(), '\nMaximum:', df['BTID'].max())

Minimum: 1 
Maximum: 1410


The values in column BTID are between -32768 to +32767, so I can use SMALLINT (https://www.postgresql.org/docs/9.1/datatype-numeric.html)

In [61]:
# check the WIND_KTS column as well
print('Minimum:', df['WIND_KTS'].min(), '\nMaximum:', df['WIND_KTS'].max())

Minimum: 10 
Maximum: 165


In [62]:
# check the PRESSURE column
print('Minimum:', df['PRESSURE'].min(), '\nMaximum:', df['PRESSURE'].max())

Minimum: 0 
Maximum: 1024


In [66]:
# check the CAT column
df['CAT'].groupby(df['CAT']).describe()

,count,unique,top,freq
CAT,,,,
E,3244,1,E,3244
H1,10658,1,H1,10658
H2,4685,1,H2,4685
H3,2607,1,H3,2607
H4,1445,1,H4,1445
H5,189,1,H5,189
L,942,1,L,942
SD,247,1,SD,247
SS,488,1,SS,488


In [67]:
# check the BASIN column
df['BASIN'].groupby(df['BASIN']).describe()

,count,unique,top,freq
BASIN,,,,
Eastern Pacific,20391,1,Eastern Pacific,20391
North Atlantic,38837,1,North Atlantic,38837


In [68]:
# check the Shape_Leng column
print('Minimum:', df['Shape_Leng'].min(), '\nMaximum:', df['Shape_Leng'].max())

Minimum: 0.0 
Maximum: 11.18034


In [161]:
cur.execute('''CREATE TABLE storm_data (
            id INTEGER PRIMARY KEY,
            date_time DATE,
            hurricane_id SMALLINT,
            hurricane_name TEXT,
            latitude DECIMAL(4,1),
            longitude DECIMAL(4,1),
            wind_speed_knots_per_second SMALLINT,
            atmosphereic_pressure SMALLINT,
            category VARCHAR(2),
            basin VARCHAR(20),
            hurricane_shape_length DECIMAL(8,6)
            )
            ''')
conn.commit()

In [160]:
#cur.execute('DROP TABLE storm_data')
#conn.commit()

In [101]:
cur.execute("SELECT * FROM storm_data")
summary = {}
counter = 0
for each in cur.description:
    dictionary = dict(zip(each._fields, each))
    summary[dictionary['name']] = dictionary

In [123]:
summary_df = pd.DataFrame(summary).drop('name')

In [124]:
summary_df

,id,date_time,hurricane_id,hurricane_name,latitude,longitude,wind_speed_knots_per_second,atmosphereic_pressure,category,basin,hurricane_shape_length
display_size,None,None,None,None,None,None,None,None,None,None,None
internal_size,4,4,2,-1,4,4,2,2,2,20,7
null_ok,None,None,None,None,None,None,None,None,None,None,None
precision,None,None,None,None,4,4,None,None,None,None,7
scale,None,None,None,None,1,1,None,None,None,None,6
type_code,23,1082,21,25,1700,1700,21,21,1043,1043,1700


Add the datatype description to make the summary easier to interpret.

In [125]:
ser = summary_df.loc['type_code'].map(type_mappings)
ser.name = 'type_description'
summary_df.append(ser)

,id,date_time,hurricane_id,hurricane_name,latitude,longitude,wind_speed_knots_per_second,atmosphereic_pressure,category,basin,hurricane_shape_length
display_size,None,None,None,None,None,None,None,None,None,None,None
internal_size,4,4,2,-1,4,4,2,2,2,20,7
null_ok,None,None,None,None,None,None,None,None,None,None,None
precision,None,None,None,None,4,4,None,None,None,None,7
scale,None,None,None,None,1,1,None,None,None,None,6
type_code,23,1082,21,25,1700,1700,21,21,1043,1043,1700
type_description,int4,date,int2,text,numeric,numeric,int2,int2,varchar,varchar,numeric


The empty table is prepared. The next step is to add the data.

## Write csv data to postgresql table

In [135]:
import io
from urllib import request
from datetime import datetime

In [132]:
response = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
reader = csv.reader(io.TextIOWrapper(response))

I started reading directly from the website, but eventually switched to just reading from the downloaded file while trying to get the rows formatted correctly.

In [155]:
with open('storm_data.csv') as f:
    reader = csv.reader(f)
    next(f)
    mogrified = []
    for row in reader:
        updated_row = [row[0]]
        date = datetime(int(row[1]), int(row[2]), int(row[3]), int(row[4][:2]), int(row[4][2:4]))
        updated_row.append(date)
        updated_row = updated_row + row[5:]
        mogrified.append(cur.mogrify("(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", updated_row).decode('utf8'))

In [162]:
mogrified_values = ",".join(mogrified)
cur.execute('INSERT INTO storm_data VALUES' + mogrified_values)
conn.commit()

In [186]:
cur.execute('SELECT * FROM storm_data')
cur.fetchall()[:3]

[(2001,
  datetime.date(1957, 8, 8),
  63,
  'NOTNAMED',
  Decimal('22.5'),
  Decimal('-140.0'),
  50,
  0,
  'TS',
  'Eastern Pacific',
  Decimal('1.140175')),
 (2002,
  datetime.date(1961, 10, 3),
  116,
  'PAULINE',
  Decimal('22.1'),
  Decimal('-140.2'),
  45,
  0,
  'TS',
  'Eastern Pacific',
  Decimal('1.166190')),
 (2003,
  datetime.date(1962, 8, 29),
  124,
  'C',
  Decimal('18.0'),
  Decimal('-140.0'),
  45,
  0,
  'TS',
  'Eastern Pacific',
  Decimal('2.102380'))]

## Create a user  
I will create a user on the Postgres database that can insert, update, and read the data but not delete.

In [168]:
cur.execute("CREATE USER vash_the_stampede WITH PASSWORD 'Typh00n' NOSUPERUSER")
conn.commit()

In [169]:
cur.execute('''REVOKE ALL ON storm_data FROM vash_the_stampede''')
conn.commit()

In [170]:
cur.execute('''GRANT SELECT ON storm_data TO vash_the_stampede''')
cur.execute('''GRANT INSERT ON storm_data TO vash_the_stampede''')
cur.execute('''GRANT UPDATE ON storm_data TO vash_the_stampede''')
conn.commit()

In [187]:
conn.close()

## Check the new user account

In [188]:
conn = psycopg2.connect(dbname='postgres', user='vash_the_stampede', password='Typh00n')
cur = conn.cursor()

In [189]:
cur.execute('SELECT * from storm_data')

In [190]:
cur.fetchall()[:3]

[(2001,
  datetime.date(1957, 8, 8),
  63,
  'NOTNAMED',
  Decimal('22.5'),
  Decimal('-140.0'),
  50,
  0,
  'TS',
  'Eastern Pacific',
  Decimal('1.140175')),
 (2002,
  datetime.date(1961, 10, 3),
  116,
  'PAULINE',
  Decimal('22.1'),
  Decimal('-140.2'),
  45,
  0,
  'TS',
  'Eastern Pacific',
  Decimal('1.166190')),
 (2003,
  datetime.date(1962, 8, 29),
  124,
  'C',
  Decimal('18.0'),
  Decimal('-140.0'),
  45,
  0,
  'TS',
  'Eastern Pacific',
  Decimal('2.102380'))]

In [176]:
cur.execute('DROP TABLE storm_data')
conn.commit()

ProgrammingError: must be owner of table storm_data


In [181]:
cur.execute('DELETE FROM storm_data')
conn.commit()

ProgrammingError: permission denied for table storm_data


In [191]:
cur.close()